In [1]:
import numpy as np
import os as os
import torch
import torch.nn as nn
import cv2
from PIL import Image

######
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [2]:
# Define the model architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x
    
model = CNN()

input_tensor = torch.randn(1, 3, 70, 70)

output = model.forward(input_tensor)
print(np.array(input_tensor).shape)

(1, 3, 70, 70)


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [3]:
def create_images(img_folder, new_folder):
    
    for dir1 in os.listdir(img_folder):
        
        if dir1 != '.DS_Store':
            
            for file in os.listdir(os.path.join(img_folder, dir1)):
                
                new_name = os.path.splitext(file)
                
                
                image_path = os.path.join(img_folder, dir1)
                new_image_path = os.path.join(new_folder, dir1)

                im1 = Image.open('{}/{}'.format(image_path, file))

                rbg_im1 = im1.convert('RGB')


                rbg_im1.save('{}/{}.jpg'.format(new_image_path, new_name[0]))
                
    return 0

# create_images('t_or_l_images', 'fixed_images')

In [4]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        if dir1 != '.DS_Store':
            for file in os.listdir(os.path.join(img_folder, dir1)):
                
                
       
                image_path= os.path.join(img_folder, dir1,  file)
                image= cv2.imread( image_path, 1)
#                 image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
                image=cv2.resize(image, (100, 100),interpolation = cv2.INTER_AREA)
                image=np.array(image)
                image = image.astype('float32')
                image /= 255 
                img_data_array.append(image)
                class_name.append(dir1)
                
    return img_data_array, class_name# extract the image array and class name
'''  temp  |  lytic   '''

dataset_X, dataset_y = create_dataset('fixed_images')


In [5]:
dataset_X = np.array(dataset_X)
dataset_y = np.array(dataset_y)

In [6]:
# img_folder = 'fixed_images'
# img_data_array, class_name = create_dataset(img_folder)

# label_dict = {'temperate': 0, 'lytic': 1}
# class_labels = [label_dict[label] for label in class_name]

# class_labels_one_hot = to_categorical(class_labels)



# X_train, X_test, y_train, y_test = train_test_split(img_data_array, class_labels_one_hot, test_size=0.2, random_state=42)

# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(2, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.fit(np.array(X_train), np.array(y_train), batch_size=22, epochs=100, verbose=0, validation_split=0.2)

# loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
# print(f"Test accuracy: {accuracy}")

# model.save('temperate_lytic_classifier.h5')


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.image import rgb_to_grayscale


img_folder = 'fixed_images'
img_data_array, class_name = create_dataset(img_folder)

label_dict = {'temperate': 0, 'lytic': 1}
class_labels = [label_dict[label] for label in class_name]

new_y = to_categorical(class_labels)

X_train, X_test, y_train, y_test = train_test_split(img_data_array, new_y, test_size=0.2, random_state=42)

# This code might break it, so remove in the future

X_train = rgb_to_grayscale(X_train)
X_test = rgb_to_grayscale(X_test)

############################

def kernelInitializer(shape, dtype=None):   
    sobel_x = tf.constant(
        [
            [-5, -4, 0, 4, 5], 
            [-8, -10, 0, 10, 8], 
            [-10, -20, 0, 20, 10], 
            [-8, -10, 0, 10, 8], 
            [-5, -4, 0, 4, 5]
        ], dtype=dtype )
    
    sobel_x = tf.reshape(sobel_x, (5, 5, 1, 1))
    
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    
    return sobel_x

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Model with more layers, batch normalization, and L1/L2 regularization
model = Sequential([
    Conv2D(64, (5, 5), activation='relu', input_shape=(100, 100, 1), strides=(2,2)),
#     BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu', strides=(2,2)),
#     BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(16, (3, 3), activation='relu', strides=(2,2)),
#     BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    
#     Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
#     Dense(232, activation='relu'),
    Dense(216, activation='relu'),
    Dense(200, activation='relu'),
    Dense(180, activation='relu'),
    Dense(150, activation='relu'),
    Dense(125, activation='relu'),
    Dense(90, activation='relu'),
    Dense(75, activation='relu'),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    
#     Dropout(0.2),
    
    Dense(2, activation='softmax')
    
])

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping
# early_stopping = EarlyStopping(
#     monitor='val_loss', patience=10, restore_best_weights=True)

training_set = datagen.flow(np.array(X_train), np.array(y_train), batch_size=25)

# Fit the model with data augmentation

# Can use 'training_set' for overfitting, or can use X_train and y_train straight up
model.fit(X_train,
          y_train,
          batch_size=20,
          epochs=200,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test))
          #
         )

print('Model before using DataGenerator:\n\n')
loss, accuracy = model.evaluate(np.array(X_train), np.array(y_train), verbose=1)
print(f"Training accuracy: {accuracy}\n")
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
print(f"Test accuracy: {accuracy}\n\n")

model.fit(training_set,
          steps_per_epoch=np.array(X_train).shape[0] // 22,
          epochs=1000,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test))
          #
         )

print('Model with after using DataGenerator:\n\n')
loss, accuracy = model.evaluate(np.array(X_train), np.array(y_train), verbose=1)
print(f"Training accuracy: {accuracy}\n")
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
print(f"Test accuracy: {accuracy}")

model.save('temperate_lytic_classifier_v2.h5')


Epoch 1/200
13/13 [==============================] - 1s 21ms/step - loss: 0.6960 - accuracy: 0.5437 - val_loss: 0.6781 - val_accuracy: 0.5938
Epoch 2/200
13/13 [==============================] - 0s 12ms/step - loss: 0.6874 - accuracy: 0.5516 - val_loss: 0.6828 - val_accuracy: 0.5938
Epoch 3/200
13/13 [==============================] - 0s 12ms/step - loss: 0.6881 - accuracy: 0.5516 - val_loss: 0.6738 - val_accuracy: 0.5938
Epoch 4/200
13/13 [==============================] - 0s 12ms/step - loss: 0.6888 - accuracy: 0.5516 - val_loss: 0.6781 - val_accuracy: 0.5938
Epoch 5/200
13/13 [==============================] - 0s 12ms/step - loss: 0.6877 - accuracy: 0.5516 - val_loss: 0.6832 - val_accuracy: 0.5938
Epoch 6/200
13/13 [==============================] - 0s 12ms/step - loss: 0.6881 - accuracy: 0.5516 - val_loss: 0.6721 - val_accuracy: 0.5938
Epoch 7/200
13/13 [==============================] - 0s 12ms/step - loss: 0.6881 - accuracy: 0.5516 - val_loss: 0.6728 - val_accuracy: 0.5938
Epoch 

13/13 [==============================] - 0s 11ms/step - loss: 0.2699 - accuracy: 0.8611 - val_loss: 1.0289 - val_accuracy: 0.6562
Epoch 59/200
13/13 [==============================] - 0s 11ms/step - loss: 0.1755 - accuracy: 0.9008 - val_loss: 1.5282 - val_accuracy: 0.6562
Epoch 60/200
13/13 [==============================] - 0s 11ms/step - loss: 0.1371 - accuracy: 0.9325 - val_loss: 2.0749 - val_accuracy: 0.5938
Epoch 61/200
13/13 [==============================] - 0s 11ms/step - loss: 0.1818 - accuracy: 0.9246 - val_loss: 1.5209 - val_accuracy: 0.6406
Epoch 62/200
13/13 [==============================] - 0s 11ms/step - loss: 0.1968 - accuracy: 0.8968 - val_loss: 1.3410 - val_accuracy: 0.5938
Epoch 63/200
13/13 [==============================] - 0s 11ms/step - loss: 0.2139 - accuracy: 0.9246 - val_loss: 1.7622 - val_accuracy: 0.5938
Epoch 64/200
13/13 [==============================] - 0s 11ms/step - loss: 0.1604 - accuracy: 0.9286 - val_loss: 1.6188 - val_accuracy: 0.6562
Epoch 65/200

13/13 [==============================] - 0s 11ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 6.6560 - val_accuracy: 0.6094
Epoch 116/200
13/13 [==============================] - 0s 12ms/step - loss: 0.0995 - accuracy: 0.9802 - val_loss: 4.3567 - val_accuracy: 0.6406
Epoch 117/200
13/13 [==============================] - 0s 11ms/step - loss: 0.2349 - accuracy: 0.9167 - val_loss: 1.8290 - val_accuracy: 0.5781
Epoch 118/200
13/13 [==============================] - 0s 12ms/step - loss: 0.1403 - accuracy: 0.9444 - val_loss: 1.5992 - val_accuracy: 0.6250
Epoch 119/200
13/13 [==============================] - 0s 12ms/step - loss: 0.1087 - accuracy: 0.9563 - val_loss: 2.0908 - val_accuracy: 0.5781
Epoch 120/200
13/13 [==============================] - 0s 12ms/step - loss: 0.0574 - accuracy: 0.9841 - val_loss: 2.5931 - val_accuracy: 0.6406
Epoch 121/200
13/13 [==============================] - 0s 12ms/step - loss: 0.0303 - accuracy: 0.9841 - val_loss: 3.3451 - val_accuracy: 0.6562
Epoch 

13/13 [==============================] - 0s 12ms/step - loss: 6.7642e-05 - accuracy: 1.0000 - val_loss: 7.8705 - val_accuracy: 0.5938
Epoch 172/200
13/13 [==============================] - 0s 13ms/step - loss: 6.1949e-05 - accuracy: 1.0000 - val_loss: 7.9453 - val_accuracy: 0.5938
Epoch 173/200
13/13 [==============================] - 0s 12ms/step - loss: 5.7889e-05 - accuracy: 1.0000 - val_loss: 8.0127 - val_accuracy: 0.5938
Epoch 174/200
13/13 [==============================] - 0s 12ms/step - loss: 5.5774e-05 - accuracy: 1.0000 - val_loss: 8.1233 - val_accuracy: 0.6094
Epoch 175/200
13/13 [==============================] - 0s 12ms/step - loss: 4.5882e-05 - accuracy: 1.0000 - val_loss: 8.2116 - val_accuracy: 0.6094
Epoch 176/200
13/13 [==============================] - 0s 12ms/step - loss: 4.0888e-05 - accuracy: 1.0000 - val_loss: 8.3022 - val_accuracy: 0.6094
Epoch 177/200
13/13 [==============================] - 0s 12ms/step - loss: 3.5791e-05 - accuracy: 1.0000 - val_loss: 8.3954 -

11/11 [==============================] - 0s 19ms/step - loss: 0.6879 - accuracy: 0.5516 - val_loss: 0.6801 - val_accuracy: 0.5938
Epoch 26/1000
11/11 [==============================] - 0s 20ms/step - loss: 0.6879 - accuracy: 0.5516 - val_loss: 0.6803 - val_accuracy: 0.5938
Epoch 27/1000
11/11 [==============================] - 0s 20ms/step - loss: 0.6883 - accuracy: 0.5516 - val_loss: 0.6817 - val_accuracy: 0.5938
Epoch 28/1000
11/11 [==============================] - 0s 19ms/step - loss: 0.6883 - accuracy: 0.5516 - val_loss: 0.6820 - val_accuracy: 0.5938
Epoch 29/1000
11/11 [==============================] - 0s 19ms/step - loss: 0.6886 - accuracy: 0.5516 - val_loss: 0.6805 - val_accuracy: 0.5938
Epoch 30/1000
11/11 [==============================] - 0s 20ms/step - loss: 0.6880 - accuracy: 0.5516 - val_loss: 0.6803 - val_accuracy: 0.5938
Epoch 31/1000
11/11 [==============================] - 0s 19ms/step - loss: 0.6878 - accuracy: 0.5516 - val_loss: 0.6815 - val_accuracy: 0.5938
Epoch 

...

11/11 [==============================] - 0s 24ms/step - loss: 0.6846 - accuracy: 0.5516 - val_loss: 0.6784 - val_accuracy: 0.5938
Epoch 755/1000
11/11 [==============================] - 0s 24ms/step - loss: 0.6870 - accuracy: 0.5516 - val_loss: 0.6840 - val_accuracy: 0.5938
Epoch 756/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6842 - accuracy: 0.5516 - val_loss: 0.6819 - val_accuracy: 0.5938
Epoch 757/1000
11/11 [==============================] - 0s 23ms/step - loss: 0.6876 - accuracy: 0.5516 - val_loss: 0.6817 - val_accuracy: 0.5938
Epoch 758/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6877 - accuracy: 0.5516 - val_loss: 0.6812 - val_accuracy: 0.5938
Epoch 759/1000
11/11 [==============================] - 0s 24ms/step - loss: 0.6879 - accuracy: 0.5516 - val_loss: 0.6811 - val_accuracy: 0.5938
Epoch 760/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6875 - accuracy: 0.5516 - val_loss: 0.6819 - val_accuracy: 0.5938


11/11 [==============================] - 0s 22ms/step - loss: 0.6845 - accuracy: 0.5516 - val_loss: 0.7010 - val_accuracy: 0.5938
Epoch 811/1000
11/11 [==============================] - 0s 26ms/step - loss: 0.6773 - accuracy: 0.5516 - val_loss: 0.6896 - val_accuracy: 0.5938
Epoch 812/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6772 - accuracy: 0.5516 - val_loss: 0.6876 - val_accuracy: 0.5938
Epoch 813/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6742 - accuracy: 0.5516 - val_loss: 0.6837 - val_accuracy: 0.5938
Epoch 814/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6768 - accuracy: 0.5516 - val_loss: 0.6835 - val_accuracy: 0.5938
Epoch 815/1000
11/11 [==============================] - 0s 24ms/step - loss: 0.6814 - accuracy: 0.5516 - val_loss: 0.6836 - val_accuracy: 0.5938
Epoch 816/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6780 - accuracy: 0.5516 - val_loss: 0.6869 - val_accuracy: 0.5938


11/11 [==============================] - 0s 23ms/step - loss: 0.6698 - accuracy: 0.5516 - val_loss: 0.6772 - val_accuracy: 0.5938
Epoch 867/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6789 - accuracy: 0.5516 - val_loss: 0.6764 - val_accuracy: 0.5938
Epoch 868/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6762 - accuracy: 0.5516 - val_loss: 0.6788 - val_accuracy: 0.5938
Epoch 869/1000
11/11 [==============================] - 0s 23ms/step - loss: 0.6699 - accuracy: 0.5516 - val_loss: 0.6809 - val_accuracy: 0.5938
Epoch 870/1000
11/11 [==============================] - 0s 24ms/step - loss: 0.6814 - accuracy: 0.5516 - val_loss: 0.6812 - val_accuracy: 0.5938
Epoch 871/1000
11/11 [==============================] - 0s 24ms/step - loss: 0.6699 - accuracy: 0.5516 - val_loss: 0.6769 - val_accuracy: 0.5938
Epoch 872/1000
11/11 [==============================] - 0s 25ms/step - loss: 0.6829 - accuracy: 0.5516 - val_loss: 0.6791 - val_accuracy: 0.5938


11/11 [==============================] - 0s 22ms/step - loss: 0.7068 - accuracy: 0.5516 - val_loss: 0.6810 - val_accuracy: 0.5938
Epoch 923/1000
11/11 [==============================] - 0s 23ms/step - loss: 0.6783 - accuracy: 0.5516 - val_loss: 0.6796 - val_accuracy: 0.5938
Epoch 924/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6831 - accuracy: 0.5516 - val_loss: 0.6823 - val_accuracy: 0.5938
Epoch 925/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6827 - accuracy: 0.5516 - val_loss: 0.6852 - val_accuracy: 0.5938
Epoch 926/1000
11/11 [==============================] - 0s 25ms/step - loss: 0.6798 - accuracy: 0.5516 - val_loss: 0.6839 - val_accuracy: 0.5938
Epoch 927/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6788 - accuracy: 0.5516 - val_loss: 0.6850 - val_accuracy: 0.5938
Epoch 928/1000
11/11 [==============================] - 0s 23ms/step - loss: 0.6726 - accuracy: 0.5516 - val_loss: 0.7010 - val_accuracy: 0.5938


11/11 [==============================] - 0s 22ms/step - loss: 0.6776 - accuracy: 0.5516 - val_loss: 0.6830 - val_accuracy: 0.5938
Epoch 979/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6822 - accuracy: 0.5516 - val_loss: 0.6824 - val_accuracy: 0.5938
Epoch 980/1000
11/11 [==============================] - 0s 23ms/step - loss: 0.6779 - accuracy: 0.5516 - val_loss: 0.6831 - val_accuracy: 0.5938
Epoch 981/1000
11/11 [==============================] - 0s 23ms/step - loss: 0.6740 - accuracy: 0.5516 - val_loss: 0.6872 - val_accuracy: 0.5938
Epoch 982/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6777 - accuracy: 0.5516 - val_loss: 0.6897 - val_accuracy: 0.5938
Epoch 983/1000
11/11 [==============================] - 0s 22ms/step - loss: 0.6749 - accuracy: 0.5516 - val_loss: 0.6944 - val_accuracy: 0.5938
Epoch 984/1000
11/11 [==============================] - 0s 24ms/step - loss: 0.6817 - accuracy: 0.5516 - val_loss: 0.6795 - val_accuracy: 0.5938


In [ ]:


shape = dataset_X.shape

print(dataset_X.shape)

dataset_X = np.reshape(dataset_X, (shape[0],shape[3],shape[1],shape[2]))

print(dataset_X.shape)

output = model.forward(torch.tensor(dataset_X))

print()

In [ ]:
torch.compile(CNN())

In [ ]:
# DO NOT RUN THIS IF YOU WANT TO KEEP KERNEL ALIVE

import tensorflow as tf
from tensorflow.keras import layers

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])